In [1]:
using BenchmarkTools
using CSV
using DataFrames
using Kraft
using StatsBase

┌ Info: Recompiling stale cache file /Users/kwatme/.julia/compiled/v1.0/Kraft/F4FiQ.ji for Kraft [5c7faf16-a6b3-11e9-0c80-d5a0766e5f8b]
└ @ Base loading.jl:1190
┌ Warning: Package Kraft does not have Distributed in its dependencies:
│ - If you have Kraft checked out for development and have
│   added Distributed as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Kraft
└ Loading Distributed into Kraft from project dependency, future warnings for Kraft are suppressed.


In [2]:
const genes = readlines("genes.txt")[2:end]

57954-element Array{String,1}:
 "MIR628"    
 "RNU6-871P" 
 "MIR626"    
 "AC012314.7"
 "GHRLOS"    
 "RNU6-35P"  
 "MIR5694"   
 "RNU6-1157P"
 "MIR6861"   
 "RNU4-85P"  
 "MIR6759"   
 "MIR4491"   
 "RNU6-1162P"
 ⋮           
 "KP420441.2"
 "KP420444.3"
 "KP420444.4"
 "KP420441.4"
 "KP420446.2"
 "KP420441.1"
 "KP420441.3"
 "KP420441.5"
 "KP420440.2"
 "KP420440.6"
 "KP420440.5"
 "KP420446.1"

In [3]:
# const gene_values = fill(
#     1 / length(genes),
#     length(genes),
# )
const gene_values = randn(length(genes))

57954-element Array{Float64,1}:
 -0.39137870416816256
 -1.7506173325737513 
 -0.8531094391341868 
  0.6439168414830988 
 -0.9260888363600662 
 -0.4111148486711711 
  0.28865923193087356
  1.0686272150792058 
  0.17220364599590424
 -0.9680620604315935 
 -0.6907934745796107 
  1.5073792766057734 
  2.754609765928508  
  ⋮                  
  2.4295831362668996 
 -0.38543983899128464
  0.25203136237746415
 -0.4913458622843178 
 -0.6181652368554985 
 -0.6115770018013353 
  0.6251230961920629 
  1.3214228889593618 
  0.28839900865235796
 -1.613516588121958  
 -0.1693204045458491 
 -0.8496479185915761 

In [4]:
const gene_x_sample = DataFrame(
    Symbol("Gene") => genes,
    Symbol("Sample 1") => gene_values,
    Symbol("Sample 2") => gene_values * 10,
)

CSV.write(
    "gene_x_sample.tsv",
    gene_x_sample,
    delim = '\t',
)

"gene_x_sample.tsv"

In [5]:
const random_gene_set_genes = sample(
    genes,
    100,
    replace = false,
)

100-element Array{String,1}:
 "RNU6-28P"  
 "HMGN2P41"  
 "CLDN23"    
 "RDH14"     
 "HNRNPA1P25"
 "AL449363.1"
 "RPL27AP5"  
 "C1orf56"   
 "AP000532.2"
 "CU207370.1"
 "ZNF723"    
 "BTF3P2"    
 "FLJ37035"  
 ⋮           
 "AL445223.1"
 "UBE2L6"    
 "BCKDHA"    
 "AC099791.1"
 "COX11"     
 "SMIM20"    
 "RNU6-812P" 
 "MT2A"      
 "MAP4K1"    
 "AC008088.1"
 "AL138916.2"
 "BMS1P22"   

In [6]:
const gene_set_genes = Kraft.read_gmt([
    "h.all.v6.2.symbols.gmt",
])

Dict{String,Array{String,1}} with 50 entries:
  "HALLMARK_REACTIVE_OXIGE… => ["GSR", "PRDX2", "TXNRD1", "SOD1", "GCLC", "CAT"…
  "HALLMARK_ANGIOGENESIS"   => ["VCAN", "POSTN", "FSTL1", "LRPAP1", "STC1", "LP…
  "HALLMARK_MYOGENESIS"     => ["ACTA1", "TNNI2", "MYL1", "TNNC1", "TNNC2", "MY…
  "HALLMARK_PEROXISOME"     => ["ABCD3", "ACOT8", "ACOX1", "ACSL1", "ECH1", "EC…
  "HALLMARK_UV_RESPONSE_DN" => ["TJP1", "NFIB", "TGFBR3", "TFPI", "MMP16", "ABC…
  "HALLMARK_NOTCH_SIGNALIN… => ["JAG1", "NOTCH3", "NOTCH2", "APH1A", "HES1", "C…
  "HALLMARK_ANDROGEN_RESPO… => ["KLK3", "KLK2", "ACSL3", "PIAS1", "CAMKK2", "NK…
  "HALLMARK_ALLOGRAFT_REJE… => ["PTPRC", "IL12B", "TGFB1", "IL12A", "CD3E", "CD…
  "HALLMARK_ESTROGEN_RESPO… => ["TFF1", "SLC9A3R1", "TPD52L1", "PRSS23", "CA12"…
  "HALLMARK_G2M_CHECKPOINT" => ["AURKA", "CCNA2", "TOP2A", "CCNB2", "CENPA", "B…
  "HALLMARK_MYC_TARGETS_V2" => ["SLC19A1", "MRTO4", "TMEM97", "RRP9", "PES1", "…
  "HALLMARK_APOPTOSIS"      => ["CASP3", "CASP9", "DFFA", "CASP

## Compute gene set enrichment with 1 sample & 1 gene set

In [7]:
Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    random_gene_set_genes,
)

([-1.72849e-5, -3.45698e-5, -5.18547e-5, -6.91396e-5, -8.64244e-5, -0.000103709, -0.000120994, -0.000138279, -0.000155564, -0.000172849  …  0.000155564, 0.000138279, 0.000120994, 0.000103709, 8.64244e-5, 6.91396e-5, 5.18547e-5, 3.45698e-5, 1.72849e-5, 1.8324e-13], 0.1961300089677972, 974.9012603937508)

In [8]:
# @benchmark Kraft.compute_gene_set_enrichment(
#     gene_values,
#     genes,
#     random_gene_set_genes,
# )

## Compute gene set enrichment with 1 sample & 1< gene sets

In [9]:
Kraft.compute_gene_set_enrichment(
    gene_values,
    genes,
    gene_set_genes,
)

Dict{String,Tuple{Union{Nothing, Array{Float64,1}},Float64,Float64}} with 50 entries:
  "HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY"   => (nothing, -0.263452, -4389.02)
  "HALLMARK_ANGIOGENESIS"                      => (nothing, 0.210108, 1188.63)
  "HALLMARK_MYOGENESIS"                        => (nothing, -0.165639, -486.563)
  "HALLMARK_PEROXISOME"                        => (nothing, -0.248928, -4187.19)
  "HALLMARK_UV_RESPONSE_DN"                    => (nothing, 0.174212, 821.587)
  "HALLMARK_NOTCH_SIGNALING"                   => (nothing, -0.253832, -5769.07)
  "HALLMARK_ANDROGEN_RESPONSE"                 => (nothing, 0.150541, 684.64)
  "HALLMARK_ALLOGRAFT_REJECTION"               => (nothing, 0.165715, 1025.78)
  "HALLMARK_ESTROGEN_RESPONSE_LATE"            => (nothing, -0.172916, -293.537)
  "HALLMARK_G2M_CHECKPOINT"                    => (nothing, -0.18107, -1723.09)
  "HALLMARK_MYC_TARGETS_V2"                    => (nothing, -0.204259, 248.016)
  "HALLMARK_APOPTOSIS"            

In [10]:
# benchmark_result = @benchmark Kraft.compute_gene_set_enrichment(
#     gene_values,
#     genes,
#     gene_set_genes,
# )

# ms_per_gene_set = round(
#     minimum(benchmark_result.times) / 1e6 / length(gene_set_genes),
#     sigdigits = 2,
# )

# println("$ms_per_gene_set ms / gene set")

# benchmark_result

## Compute gene set enrichment with 1< samples & 1< gene sets

In [11]:
statistic = "ks"

"ks"

In [12]:
Kraft.compute_gene_set_enrichment(
    gene_x_sample,
    gene_set_genes,
    statistic,
)

,Gene Set,Sample 1,Sample 2
,String,Float64,Float64
1,HALLMARK_REACTIVE_OXIGEN_SPECIES_PATHWAY,-0.263452,-0.263452
2,HALLMARK_ANGIOGENESIS,0.210108,0.210108
3,HALLMARK_MYOGENESIS,-0.165639,-0.165639
4,HALLMARK_PEROXISOME,-0.248928,-0.248928
5,HALLMARK_UV_RESPONSE_DN,0.174212,0.174212
6,HALLMARK_NOTCH_SIGNALING,-0.253832,-0.253832
7,HALLMARK_ANDROGEN_RESPONSE,0.150541,0.150541
8,HALLMARK_ALLOGRAFT_REJECTION,0.165715,0.165715
9,HALLMARK_ESTROGEN_RESPONSE_LATE,-0.172916,-0.172916


In [13]:
# @benchmark gene_set_x_sample = Kraft.compute_gene_set_enrichment(
#     gene_x_sample,
#     gene_set_genes,
#     statistic,
# )

## GSEA

In [42]:
output_directory_path = "gsea_output"

gene_set_x_sample = Kraft.gsea(
    "gene_x_sample.tsv",
    [
#         "h.all.v6.2.symbols.gmt",
        "c2.all.v6.2.symbols.gmt",
    ],
    output_directory_path;
    statistic = statistic,
)

gene_set_x_sample = CSV.read(joinpath(
    output_directory_path,
    "gene_set_x_sample.tsv",
))

,Gene Set,Sample 1,Sample 2
,String,Float64,Float64
1,REACTOME_EXTRACELLULAR_MATRIX_ORGANIZATION,-0.256298,-0.256298
2,GARCIA_TARGETS_OF_FLI1_AND_DAX1_UP,0.216219,0.216219
3,SIMBULAN_PARP1_TARGETS_UP,-0.266885,-0.266885
4,WEI_MYCN_TARGETS_WITH_E_BOX,0.184473,0.184473
5,PID_RET_PATHWAY,-0.278729,-0.278729
6,MARIADASON_RESPONSE_TO_CURCUMIN_SULINDAC_7,0.350459,0.350459
7,REACTOME_GPCR_LIGAND_BINDING,-0.174432,-0.174432
8,KEGG_CYTOSOLIC_DNA_SENSING_PATHWAY,-0.300544,-0.300544
9,HOSHIDA_LIVER_CANCER_SUBCLASS_S3,0.175791,0.175791


In [43]:
gene_sets = gene_set_x_sample[:, Symbol("Gene Set")]

gene_set_combined_x_sample = DataFrame(Dict(n => Array{t, 1}() for (n, t) in zip(
    names(gene_set_x_sample),
    eltypes(gene_set_x_sample),
)))

for gene_set in gene_sets
    
    if endswith(
        gene_set,
        "_DN",
    ) || endswith(
        gene_set,
        "_UP",
    )
        
        gene_set_combined = gene_set[1:end - 3]
        
        if gene_set_combined in gene_set_combined_x_sample[:, Symbol("Gene Set")]
            
            continue
            
        end
        
        gene_set_dn = "$(gene_set_combined)_DN"
        
        gene_set_up = "$(gene_set_combined)_UP"
        
        dn_values = gene_set_x_sample[gene_sets .== gene_set_dn, 2:end]
        
        up_values = gene_set_x_sample[gene_sets .== gene_set_up, 2:end]
        
        if size(
            dn_values,
            1,
        ) == 0 || size(
            up_values,
            1,
        ) == 0
            
            continue
            
        end
        
        combined_values = up_values .- dn_values
         
        insertcols!(
            combined_values,
            1,
            Symbol("Gene Set") => gene_set_combined,
        )
        
        push!(
            gene_set_combined_x_sample,
            combined_values[1, :],
        )
        
    end
        
end

sort(
    vcat(
        gene_set_x_sample,
        gene_set_combined_x_sample,
    ),
    Symbol("Gene Set"),
)

,Gene Set,Sample 1,Sample 2
,String,Float64,Float64
1,ABBUD_LIF_SIGNALING_1,-0.233144,-0.233144
2,ABBUD_LIF_SIGNALING_1_DN,0.42417,0.42417
3,ABBUD_LIF_SIGNALING_1_UP,0.191026,0.191026
4,ABBUD_LIF_SIGNALING_2,0.0211388,0.0211388
5,ABBUD_LIF_SIGNALING_2_DN,-0.333288,-0.333288
6,ABBUD_LIF_SIGNALING_2_UP,-0.31215,-0.31215
7,ABDELMOHSEN_ELAVL4_TARGETS,0.375821,0.375821
8,ABDULRAHMAN_KIDNEY_CANCER_VHL,-0.273401,-0.273401
9,ABDULRAHMAN_KIDNEY_CANCER_VHL_DN,-0.34352,-0.34352
